In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import neurokit2 as nk

sys.path.append('..')
from util import standardize, normalize, load_base_data

import warnings
warnings.simplefilter("ignore")

plt.rcParams['figure.figsize'] = [20, 5]

In [2]:
DATA_PATH = "../data"
SAMPLING_RATE = 300

In [3]:
X_train, y_train, X_test = load_base_data(data_path=DATA_PATH, just_train=False, flip=True, normal=False)

FileNotFoundError: [Errno 2] No such file or directory: '../data/base/base/X_train.csv'

# Sandbox

In [4]:
i = 10

signal = X_train.loc[i].dropna().to_numpy(dtype='float32')
signals, info = nk.ecg_process(signal, sampling_rate=SAMPLING_RATE)
rpeaks = info["ECG_R_Peaks"]
cleaned_signal = signals["ECG_Clean"]
plt.plot(signal)
plt.show()


#_, waves_peak = nk.ecg_delineate(cleaned_signal, rpeaks, sampling_rate=SAMPLING_RATE, method="peak")


NameError: name 'X_train' is not defined

In [5]:
def get_feature_mean_std(points, signal):
    vals = []
    for i in [p for p in points if p is not np.nan]:
        vals.append(signal[i])
    return np.mean(vals), np.std(vals)

def generate_feature_vector(r_peaks, signal, waves_peak):
    feature = []
    r_peaks_mean, r_peaks_std = get_feature_mean_std(r_peaks, signal)
    feature.append(r_peaks_mean)
    feature.append(r_peaks_std)
    for key in waves_peak.keys():
        mean, std = get_feature_mean_std(waves_peak[key], signal)
        feature.append(mean)
        feature.append(std)
    return np.array(feature)

def generate_RPQST_features(data):
    feature_names = [
        'r_peaks_mean', 'r_peaks_std', 
        'p_peaks_mean', 'p_peaks_std', 'q_peaks_mean', 'q_peaks_std',
        's_peaks_mean', 's_peaks_std', 't_peaks_mean', 't_peaks_std',
        'p_onsets_mean', 'p_onsets_std', 't_offsets_mean', 't_offsets_std']
    feature_vecs = []
    indices = []
    for i in tqdm(range(len(data))):
        signal = data.loc[i].dropna().to_numpy(dtype='float32')
#         r_peaks = ecg.engzee_segmenter(signal, SAMPLING_RATE)['rpeaks']
        feature_vector = None

        try:
            signals, info = nk.ecg_process(signal, sampling_rate=SAMPLING_RATE)
            rpeaks = info["ECG_R_Peaks"]
            cleaned_signal = signals["ECG_Clean"]

            

            _, waves_peak = nk.ecg_delineate(cleaned_signal, rpeaks, sampling_rate=SAMPLING_RATE, method="peak")

            feature_vector = generate_feature_vector(rpeaks, cleaned_signal, waves_peak)
            
        except:
            pass
        
        if feature_vector is None:
            feature_vector = [np.nan]*len(feature_names)
        feature_vecs.append(feature_vector)
        indices.append(i)
        
    return feature_names, np.array(feature_vecs), indices

In [6]:
feature_names, features, indices = generate_RPQST_features(X_train)

NameError: name 'generate_RPQST_features' is not defined

In [ ]:
df = pd.DataFrame(data=features, columns=feature_names)
new_data = df.to_numpy()
with open("X_train.csv", "w") as f:
    f.write("id,"+",".join(feature_names) + "\n")
    for i, d in zip(indices, new_data):
        f.write(f"{str(i)},"+",".join([str(x) for x in d])+"\n")

In [ ]:
feature_names, features, indices = generate_RPQST_features(X_test)

In [ ]:
df = pd.DataFrame(data=features, columns=feature_names)
new_data = df.to_numpy()
with open("X_test.csv", "w") as f:
    f.write("id,"+",".join(feature_names) + "\n")
    for i, d in zip(indices, new_data):
        f.write(f"{str(i)},"+",".join([str(x) for x in d])+"\n")